In [ ]:
import os
from gtts import gTTS
import pygame
from io import BytesIO
import speech_recognition as sr
import openai

class Speech():
    def __init__(self) -> None:
        pygame.init()

    def speak(self, text:str, lang:str) -> None:
        tts = gTTS(text=text, lang=lang)
        fp = BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)
        pygame.mixer.init()
        pygame.mixer.music.load(fp)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

    def speak2(self, text:str, lang:str) -> None:
        myobj = gTTS(text=text, lang=lang, slow=False)
        myobj.save('temp.mp3')
        os.system('mpg123 temp.mp3')
        os.remove('temp.mp3')

    def listen(self, lang:str) -> str:
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Parla pure...")
            audio_text = r.listen(source)
            print("Ok")
            try:
                return str(r.recognize_google(audio_text, language=lang))
            except:
                return "Errore..."


class Dialogue():
    def __init__(self) -> None:
        openai.organization = os.getenv('OPENAI_ORGANIZATION')
        openai.api_key = os.getenv('OPENAI_API_KEY')

    def get_reply(self, prompt:str) -> str:
        messages = [{'role':'user', 'content':prompt}]
        completion = openai.ChatCompletion.create( # genera la risposta
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=512,
            n=1,
            stop=None,
            temperature=0.5
        )
        return completion["choices"][0]["message"]["content"] # type: ignore



speech = Speech()
dialogue = Dialogue()
q_text = speech.listen('it')
print(q_text)
a_text = dialogue.get_reply(q_text)
speech.speak(a_text, 'it')
print(a_text)